In [1]:
import json
from google.colab import drive
from sklearn.model_selection import train_test_split
import pandas as pd
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.4MB/s 


In [2]:
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Kindle_rating_predict"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
data_clas_save.pkl	    results_1
data_lm_export_1000.pkl     reviews_Kindle_Store_5.json
data_lm_export.pkl	    test_1000.csv
ludwig_api.ipynb	    train_1000.csv
model_definition_bert.yaml  train_1000.hdf5
model_definition.yaml	    train_1000.json
models			    train.csv
my_ULMfit.ipynb		    uncased_L-12_H-768_A-12
my_ULMfit.py		    uncased_L-12_H-768_A-12.zip
results			    wwm_uncased_L-24_H-1024_A-16
results_0


In [0]:
path="/content/drive/My Drive/Kindle_rating_predict"

In [0]:
reviews = []
file = "/content/drive/My Drive/Kindle_rating_predict/reviews_Kindle_Store_5.json"
for line in open(file, 'r'):
    reviews.append(json.loads(line))

In [0]:
reviews_df = pd.DataFrame(reviews)

In [6]:
train, test = train_test_split(reviews_df, test_size=0.15, random_state=42, shuffle=True)
train.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
208012,B007SH90RM,"[0, 0]",5.0,"This was a good one, let's start by saying tha...","05 13, 2014",A2GU8WAUL1GREZ,"Veritas Vincit ""Bill""",Deep Mystery and Masterful Storytelling,1399939200
590910,B00E123W58,"[0, 0]",5.0,I was gifted my copy of RUNE II in exchange fo...,"07 23, 2013",A1RUT4WFFKVI1I,Anne Nelson,What a explosion as it hit my kindle!!!!,1374537600
563433,B00DKB3LKM,"[0, 0]",5.0,I really liked this book. Another great book i...,"06 26, 2013",A30GJ3BNSILI9D,Bill McBride,A fact fills story,1372204800
375492,B00AM0WQD2,"[1, 1]",5.0,Great information for creating gifts! I have m...,"12 13, 2012",A3KBGY56U6MPA3,this is great,Definitely Recommended!,1355356800
293305,B0094B6QQ8,"[0, 0]",2.0,Very short story...not credible at all...and j...,"09 14, 2013",A1JWFH1A4XT5PU,Diana Galan,Bah,1379116800


In [0]:
train_sm = pd.concat([train['overall'], train['reviewText']], axis=1)
test_sm = pd.concat([test['overall'], test['reviewText']], axis=1)

In [0]:
# Get rid of reviews with no text
noDataTr = train_sm[train_sm['reviewText'] == ""].index
train_sm.drop(noDataTr, inplace=True)
noDataTe = test_sm[test_sm['reviewText'] == ""].index
test_sm.drop(noDataTe, inplace=True)

In [0]:
# Restrict our data set 
#train_sm = train_sm.sample(frac=0.3, random_state=42)
train_sm = train_sm.sample(n=1000, random_state=42)
test_sm = test_sm.sample(n=200, random_state=42)

In [0]:
my_dict = {1.0: "one star", 2.0: "two stars", 3.0: "three stars", 4.0: "four stars", 5.0: "five stars"}
train_sm = train_sm.replace({"overall": my_dict})
test_sm = test_sm.replace({"overall": my_dict})

In [11]:
!apt-get install libgmp3-dev
!pip install ludwig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgmp-dev libgmpxx4ldbl
Suggested packages:
  gmp-doc libgmp10-doc libmpfr-dev
The following NEW packages will be installed:
  libgmp-dev libgmp3-dev libgmpxx4ldbl
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 327 kB of archives.
After this operation, 1,689 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmpxx4ldbl amd64 2:6.1.2+dfsg-2 [8,964 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmp-dev amd64 2:6.1.2+dfsg-2 [316 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgmp3-dev amd64 2:6.1.2+dfsg-2 [1,996 B]
Fetched 327 kB in 1s (433 kB/s)
Selecting previously unselected package libgmpxx4ldbl:amd64.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../libgmpxx4ldbl_2%3a6.1.2+dfsg

In [12]:
from ludwig.api import LudwigModel

# train a model
model_definition = {'input_features': [{'name': 'reviewText', 'type': 'text', 'encoder': 'bert', 'config_path': '/content/drive/My Drive/Kindle_rating_predict/uncased_L-12_H-768_A-12/bert_config.json', 
                                      'checkpoint_path': '/content/drive/My Drive/Kindle_rating_predict/uncased_L-12_H-768_A-12/bert_model.ckpt', 'do_lower_case': 'True',
                                     'preprocessing': {'word_tokenizer': 'bert', 'word_vocab_file': '/content/drive/My Drive/Kindle_rating_predict/uncased_L-12_H-768_A-12/vocab.txt', 'padding_symbol': '[PAD]',
                                                    'unknown_symbol': '[UNK]'}, 'reduce_output': 'True'}], 'output_features': [{'name': 'overall', 'type': 'category'}]}

#print(model_definition)

model = LudwigModel(model_definition)
train_stats = model.train(train_sm)

# or load a model
#model = LudwigModel.load(model_path)





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





ResourceExhaustedError: ignored

In [0]:
# obtain predictions
predictions = model.predict(test_sm)

model.close()